In [ ]:
import numpy as np, pandas as pd
from scipy.sparse import coo_matrix
from implicit.bpr import BayesianPersonalizedRanking
import recometrics
from poismf import PoisMF
from pandas.api.types import CategoricalDtype
from scipy import sparse
from sklearn.model_selection import train_test_split
import recmetrics
import tqdm


In [37]:
df = pd.read_csv("netflix_user_movie.csv",sep=",",)


In [38]:
display(df)

,user_id,movie_id,rating,timestamp
0,2480084,16,1,1008720000
1,1794216,16,1,995673600
2,1907667,16,1,1109548800
3,1645486,16,1,949622400
4,1507459,16,1,1066348800
...,...,...,...,...
3443245,1441086,17764,1,1013212800
3443246,1788756,17764,1,1034726400
3443247,2381902,17764,1,1009843200
3443248,2082452,17764,1,1009584000


In [39]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


In [40]:
user_positives = train_df.groupby('user_id')['movie_id'].apply(set).to_dict()


In [41]:
display(user_positives)

KeyboardInterrupt: 

In [ ]:
train_ratings_matrix = train_df.pivot(index='user_id', columns='movie_id', values='rating')
test_ratings_matrix = test_df.pivot(index='user_id', columns='movie_id', values='rating')

# rating_matrix = ratings_matrix.reindex(sorted(ratings_matrix.index, key=int), axis=0)
# rating_matrix = rating_matrix.reindex(sorted(rating_matrix.columns, key=int), axis=1)

In [ ]:
display(train_ratings_matrix)
display(test_ratings_matrix)

movie_id,16,18,29,44,45,46,47,55,57,77,...,17740,17742,17746,17747,17754,17756,17758,17762,17763,17764
user_id,,,,,,,,,,,,,,,,,,,,,
684,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
1333,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0
2307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2757,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2648298,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2648853,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2649097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


movie_id,16,18,29,44,45,46,47,55,57,77,...,17740,17742,17746,17747,17754,17756,17758,17762,17763,17764
user_id,,,,,,,,,,,,,,,,,,,,,
684,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1333,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2757,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2648298,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2648853,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2649097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
users = train_df["user_id"].unique()
movies = train_df["movie_id"].unique()
shape = (len(users), len(movies))

# Create indices for users and movies
user_cat = CategoricalDtype(categories=sorted(users), ordered=True)
movie_cat = CategoricalDtype(categories=sorted(movies), ordered=True)
user_index = train_df["user_id"].astype(user_cat).cat.codes
movie_index = train_df["movie_id"].astype(movie_cat).cat.codes

# Conversion via COO matrix
coo = sparse.coo_matrix((train_df["rating"], (user_index, movie_index)), shape=shape)
csr = coo.tocsr()
#https://hippocampus-garden.com/pandas_sparse/

In [ ]:
bpr = BayesianPersonalizedRanking(factors=19, learning_rate=0.001,
                                  regularization=0.001, dtype=np.float64,
                                  iterations=100)

In [ ]:
bpr.fit(csr)


  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
pred = np.matmul(bpr.user_factors,bpr.item_factors.T)

In [ ]:
from recsys_metrics import *
import torch
# https://github.com/zuoxingdong/recsys_metrics?tab=readme-ov-file#Citation

In [ ]:
true = torch.tensor(test_ratings_matrix.fillna(0).values)

In [ ]:
# pred[pred < 0.49] = 0
# pred[pred > 0.5] = 1

# print(pred)

In [ ]:
predv = torch.tensor(pred)

In [ ]:
display(true)
display(predv)

tensor([[0., 0., 0.,  ..., 0., 0., 1.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)

tensor([[-0.3430,  0.2164, -0.1044,  ...,  0.8020, -0.1713,  1.0034],
        [-0.6038,  0.7360,  0.0923,  ...,  0.4012,  0.3537,  0.6909],
        [-0.4168, -0.2397, -0.1389,  ...,  0.1105, -0.7018,  1.0391],
        ...,
        [-0.9408,  1.1582,  0.0373,  ...,  0.8793,  0.8762,  1.4211],
        [-0.0259, -0.1608, -0.1959,  ...,  0.5799, -1.1630,  0.9843],
        [ 0.0667, -0.7110, -0.2708,  ...,  0.9026, -1.5791,  1.1057]],
       dtype=torch.float64)

In [ ]:
def calculate_metrics(pred_df, user_postivies, k_list = [10,50,100]):
  metrics = {
    'MAR@ALL': [],
    'MAR@1000': [],
    'MRR': [],
  }


  grouped = pred_df.groupby('user')

  for user, user_data in tqdm.tqdm(grouped, desc="Processing Users"):
    positives = user_postivies.get(user, set())
    if len(positives) == 0:
      continue

    sorted_data = user_data.sort_values(by='score', ascending=False)
    all_items = sorted_data['item'].tolist()

    relevant_ranks1k = []
    relevant_ranks_all=[]
    precisions = []
    hit_at_ranks = None
    for rank, item in enumerate(all_items, start=1):
      if item in positives:
        relevant_ranks1k.append(rank)
    #calculate relevant
    for rank, item in enumerate(all_items, start=1):
      if item in positives:
        relevant_ranks_all.append(rank)
        precisions.append(len(positives) / rank)

        if hit_at_ranks is None:
            hit_at_ranks = rank


    rr = 1.0 / hit_at_ranks if hit_at_ranks is not None else 0.0
    metrics['MRR'].append(rr)

    if relevant_ranks_all:
      metrics['MAR@ALL'].append(np.mean(relevant_ranks_all))
    if relevant_ranks1k:
      metrics['MAR@1000'].append(np.mean(relevant_ranks_all))
      
    k = 1000  # or pass as argument
    all_items = sorted_data['item'].tolist()[:k]
  
  final = {
    'MAR@ALL': np.mean(metrics['MAR@ALL']),
    'MAR@1000': np.mean(metrics['MAR@1000']),
    'MRR': np.mean(metrics['MRR']),
  }

  return final

In [ ]:
def metricStuff(pred,true,user_positive):
    print("NDCG@10",normalized_dcg(pred, true ,k=10))
    print("MRR@1000",mean_reciprocal_rank(pred, true,k=1000))
    print("MRR@all",mean_reciprocal_rank(pred, true))
    # marData = pred.detach().numpy()
    # marDF = pd.DataFrame(marData)
    # marDF.index.name = "user"
    # marDF.columns.name = "items"
    # marDF = marDF.stack().reset_index()
    # marDF.columns = ['user', 'item', 'score']
    marData = pred.detach().numpy()

    # Only include non-zero scores (or > threshold if you like)
    non_zero_indices = np.argwhere(marData > 0)

    marDF = pd.DataFrame({
        'user': non_zero_indices[:, 0],
        'item': non_zero_indices[:, 1],
        'score': marData[non_zero_indices[:, 0], non_zero_indices[:, 1]]
    })
    print("MAR",calculate_metrics(marDF,user_positive))
    print("MAP",mean_average_precision(pred, true))
    print("P@50",precision(pred, true,k=50))
    print("R@50",recall(pred, true,k=50))
    

In [ ]:
metricStuff(predv,true,user_positives)

NDCG@10 tensor(0.1672)
MRR@1000 tensor(0.3592)
MRR@all tensor(0.3592)


Processing Users: 100%|██████████| 6163/6163 [00:02<00:00, 3071.54it/s]


MAR {'MAR@ALL': np.float64(1986.4522936308226), 'MAR@1000': np.float64(1986.4522936308226), 'MRR': np.float64(0.07053081457038708)}
MAP tensor(0.1220)
P@50 tensor(0.1623)
R@50 tensor(0.0911)
